In [ ]:
# Depends on
import random
import math

# Benchmark
import time

**Decode Binary Function**

This function utilizes binary representation (see Pokok Bahasan 4 p. 14).

Returns the value of given binary bit

Arguments:

1. ***bit***, list, required
2. ***lower***, integer, optional, default -5
3. ***upper***, integer, optional, default 5

In [ ]:
def decodeBinary(bit: list, lower: int = -5, upper: int = 5) -> float:
  bitLength = len(bit)
  summation = [2**-i for i in range(1, bitLength + 1)]
  constant = sum([bit[i] * summation[i] for i in range(bitLength)])

  return lower + ((upper - lower) / sum(summation) * constant)

**Heuristic Function**

Returns the value of given x and y of given heuristic function

Arguments:

1. ***x***, integer, required
2. ***y***, integer, required

**Fitness Function**

This function utilizes fitness function (see Pokok Bahasan 4 p. 19).

Returns the value of given heuristic function its fitness value

Arguments:

1. ***x***, integer, required
2. ***y***, integer, required

In [ ]:
def f(x: int, y: int) -> float:
  def h(x: int, y: int) -> float:
    return ((math.cos(x) + math.sin(y))**2) / (x**2) + (y**2)

  a = random.uniform(0.01, 0.1)
  return 1 / (h(x, y) + a)

**Compute Chromosome Function**

Returns the computed value in a dictionary of a chromosome

Arguments:

1. ***x***, list of x bit, required 
2. ***y***, list of y bit, required 

In [ ]:
def computeChromosome(x: list, y: list) -> dict:
  xPhenotype = decodeBinary(x)
  yPhenotype = decodeBinary(y)

  return {
      "chromosome": x + y,
      "fitness": f(xPhenotype, yPhenotype),
      "genotype": {
          "x": x,
          "y": y
      },
      "phenotype": {
          "x": xPhenotype,
          "y": yPhenotype
      }
  }

**Build Chromosome Function**

Returns the computed value of a recently built chromosome

Arguments:

1. ***length***, integer, optional, default 10

In [ ]:
def buildChromosome(GENOTYPE_LENGTH: int = 10) -> dict:
  x = random.choices([0, 1], k = GENOTYPE_LENGTH)
  y = random.choices([0, 1], k = GENOTYPE_LENGTH)

  return computeChromosome(x, y)

**Parent Selection Function**

This function utilizes Tournament Selection (see Pokok Bahasan 4 p. 27).

Returns 1 parent in a tournament pool with highest fitness value

```
Example returned value:
{
    "chromosome": 010111101000,
    "fitness": 95.123456789,
    "genotype": {
        "x": 010111,
        "y": 101000
    },
    "phenotype": {
        "x": 2,
        "y": -1
    }
}
```

Arguments:

1. ***populations***, list of chromosome, required
2. ***poolSize***, integer, optional, default 5

In [ ]:
def selectParent(populations: list, PARENT_POOL_SIZE: int = 5) -> dict:
  pool = random.sample(populations, PARENT_POOL_SIZE)

  return sorted(pool, key = lambda x: x["fitness"], reverse = True)[0]

**Crossover Function**

This function utilizes 1-point recombination (see Pokok Bahasan 4 p. 32).

Returns array of crossed chromosome between x and y (if the probability). Otherwise returns x and y

```
Example of a given parents:
[0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1],
[0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]

Intersection Point: 2

Example returned value:
[
  {
    'chromosome': [0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1], 
    'fitness': 1.7603062053954148, 
    'genotype': {
      'x': [0, 0, 0, 1, 0, 0, 1, 1, 0, 1], 
      'y': [1, 0, 0, 1, 0, 0, 0, 0, 0, 1]
    }, 
    'phenotype': {
      'x': -4.247311827956989, 
      'y': 0.6402737047898341
    }
  }, 
  {
    'chromosome': [0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    'fitness': 1.73262076789398, 
    'genotype': {
      'x': [0, 0, 0, 0, 1, 1, 0, 0, 0, 1], 
      'y': [1, 0, 0, 0, 1, 0, 0, 0, 1, 0]
    }, 
    'phenotype': {
      'x': -4.521016617790812, 
      'y': 0.3372434017595305
    }
  }
]
```



Arguments:

1. ***x***, dictionary of a chromosome, required
2. ***y***, dictionary of a chromosome, required
3. ***length***, integer, optional, default 10 (a x or y bit length)
4. ***rate***, float, optional, default 0.7 

In [ ]:
def crossover(x: dict, y: dict, GENOTYPE_LENGTH: int = 10, CROSSOVER_RATE = 0.7) -> list:
  chromosomeLength = 2 * GENOTYPE_LENGTH
  divider = chromosomeLength // 2
  intersectionPoint = random.randint(0, chromosomeLength - 1)

  if random.uniform(0, 1) <= CROSSOVER_RATE:
    childs = {
      "x": y["chromosome"][:intersectionPoint] + x["chromosome"][intersectionPoint:],
      "y": x["chromosome"][:intersectionPoint] + y["chromosome"][intersectionPoint:]
    }

    return [computeChromosome(childs[i][:divider], childs[i][divider:]) for i in childs.keys()]
  else:
    return [x, y]

**Mutate Function**

This function utilizes Mutation for Binary Representation (see Pokok Bahasan 4 p. 49).

Returns a mutated chromosome child dictionary

Arguments:

1. ***x***, dictionary of a chromosome, required
2. ***length***, integer, optional, default 10 (a x or y bit length)
3. ***rate***, float, optional, default 0.01 (2 in 20 (the chromosome length))

In [ ]:
def mutate(x: dict, GENOTYPE_LENGTH: int = 10, MUTATION_RATE: int = 0.01) -> dict:
  chromosomeLength = 2 * GENOTYPE_LENGTH
  divider = chromosomeLength // 2

  for i in range(chromosomeLength):
    if random.uniform(0, 1) <= MUTATION_RATE:
      if x["chromosome"][i] == 0:
        x["chromosome"][i] = 1
      else:
        x["chromosome"][i] = 0

  xBit = x["chromosome"][:divider]
  yBit = x["chromosome"][divider:]

  return computeChromosome(xBit, yBit)

**Survivor Selection Function**

This function utilizes Generational Survivor Selection (see Pokok Bahasan 4 p. 63).

Returns a N (half from crossover, and another half from mutated) sized new populations. Mutated population always sacrified for elitism.

Arguments:

1. ***childs***, list, required
2. ***populations***, list, required
3. ***length***, integer, optional, default 10
4. ***crossoverRate***, float, optional, default 0.7
5. ***mutationRate***, float, optional, default 0.1
6. ***parentPoolSize***, integer, optional, default 5
7. ***matingPoolSize***, integer, optional, default 25

In [ ]:
def survivorSelection(childs: list, populations: list, GENOTYPE_LENGTH = 10, CROSSOVER_RATE = 0.7, MUTATION_RATE = 0.01, PARENT_POOL_SIZE = 5, MATING_POOL_SIZE = 25) -> list:
  for child in childs:
    if child["fitness"] > populations[-1]["fitness"]:
      populations[-1] = child

  # Elitism
  bestIndividual = populations[0]

  pool = [[selectParent(populations, PARENT_POOL_SIZE) for j in range(2)] for i in range(MATING_POOL_SIZE)]
  crossoverPopulation = [crossover(pool[i][0], pool[i][1], GENOTYPE_LENGTH, CROSSOVER_RATE) for i in range(MATING_POOL_SIZE)]

  newPopulation = [mutate(crossoverPopulation[i][0], GENOTYPE_LENGTH, MUTATION_RATE) for i in range(MATING_POOL_SIZE)] + [mutate(crossoverPopulation[i][1], GENOTYPE_LENGTH, MUTATION_RATE) for i in range(MATING_POOL_SIZE)]
  newPopulation.append(bestIndividual)

  return sorted(newPopulation, key = lambda x: x["fitness"], reverse = True)

**Generate Population Function**

Returns a list containing genetic information in specified min and max domain

```
Example returned value:
[{
      "chromosome": 101000010111,
      "fitness": 0.0123456789,
      "genotype": {
          "x": 101000,
          "y": 010111
      },
      "phenotype": {
          "x": -1,
          "y": 2
      }
}, ..., {
      "chromosome": 010111101000,
      "fitness": 0.0123456789,
      "genotype": {
          "x": 010111,
          "y": 101000
      },
      "phenotype": {
          "x": 2,
          "y": -1
      }
}]
```

Arguments:

1. ***length***, integer, optional, default 10
2. ***size***, integer, optional, default 50 (number of population)

In [ ]:
def generatePopulation(GENOTYPE_LENGTH: int = 10, POPULATION_SIZE: int = 50) -> list:
  return [buildChromosome(GENOTYPE_LENGTH) for i in range(POPULATION_SIZE)]

In [ ]:
def main():
  start = time.time()
  print("Hello, mom!")

  # Constant
  POPULATION_SIZE = 50
  PARENT_POOL_SIZE = 5
  GENOTYPE_LENGTH = 10
  CROSSOVER_RATE = 0.7
  MUTATION_RATE = 0.01
  MATING_POOL_SIZE = POPULATION_SIZE // 2
  MAX_GENERATION = 10000

  # Initial Population
  populations = sorted(generatePopulation(GENOTYPE_LENGTH, POPULATION_SIZE), key = lambda x: x["fitness"], reverse = True)

  # Evaluation Variable
  generation = 0
  bestIndividual = populations[0]
  MIN_THRESHOLD = 99.0

  while bestIndividual["fitness"] <= MIN_THRESHOLD and generation < MAX_GENERATION:
    generation = generation + 1

    x = selectParent(populations, PARENT_POOL_SIZE)
    y = selectParent(populations, PARENT_POOL_SIZE)

    xChild, yChild = crossover(x, y, GENOTYPE_LENGTH, CROSSOVER_RATE)

    xMutant = mutate(xChild, GENOTYPE_LENGTH, MUTATION_RATE)
    yMutant = mutate(yChild, GENOTYPE_LENGTH, MUTATION_RATE)

    childs = [xMutant, yMutant]

    survivor = survivorSelection(childs, populations, GENOTYPE_LENGTH, CROSSOVER_RATE, MUTATION_RATE, PARENT_POOL_SIZE, MATING_POOL_SIZE)
    populations = survivor

    bestIndividual = populations[0]

  print(f"Generation", generation)

  if bestIndividual["fitness"] >= MIN_THRESHOLD:
    print("Evolution stopped because a fitness above", MIN_THRESHOLD, "have been found")
  elif generation >= MAX_GENERATION:
    print("Evolution stopped because generation have exceed", MAX_GENERATION)

  print("\nAll time best value:")
  print(f"Chromosome:", ''.join([str(bit) for bit in bestIndividual["chromosome"]]))
  print(f"Fitness:", bestIndividual["fitness"])
  print(f"Phenotype (X):", bestIndividual["phenotype"]["x"])
  print(f"Phenotype (Y):", bestIndividual["phenotype"]["y"])

  print(f"\nElapsed: ", time.time() - start)

In [ ]:
if __name__ == "__main__":
  main()

Hello, mom!
Generation 457
Evolution stopped because a fitness above 99.0 have been found

All time best value:
Chromosome: 00000111111000000000
Fitness: 99.53636667320501
Phenotype (X): -4.696969696969697
Phenotype (Y): 0.004887585532746819

Elapsed:  1.0598375797271729
